In [11]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import sys
# Set root and paths
ROOT_PATH = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
sys.path.append(ROOT_PATH)

from Training.Helper.dataPreprocessing import (
    add_time_features, add_lagged_features, add_rolling_features,
    sklearn_fit_transform, prepare_dataloader, rank_features_ccf,
    TRAIN_DATA_PATH_1990S
)
from Models.LSTM import LSTM

def create_direct_delta_sequences(X, y, seq_len=36, horizon=12):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_len - horizon):
        X_seq.append(X[i:i + seq_len])
        base = y[i + seq_len - 1]
        future = y[i + seq_len: i + seq_len + horizon]
        delta = future - base
        y_seq.append(delta)
    return np.array(X_seq), np.array(y_seq)

# === CONFIG ===
SEQ_LEN = 36
HORIZON = 12
BATCH_SIZE = 16
EPOCHS = 100
PATIENCE = 10
LR = 1e-3
TOP_K_FEATURES = 30
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [12]:
# === LOAD & FEATURE ENGINEERING ===
df = pd.read_csv(TRAIN_DATA_PATH_1990S)
df["ds"] = pd.to_datetime(df["observation_date"], format="%m/%Y")
df = df.rename(columns={"fred_PCEPI": "y_original"})

df = add_time_features(df, "ds")
for k in [1, 2, 3, 4]:
    df[f"sin_{k}"] = np.sin(2 * np.pi * k * df["month"] / 12)
    df[f"cos_{k}"] = np.cos(2 * np.pi * k * df["month"] / 12)
df["pct_change"] = df["y_original"].pct_change()
df["momentum"] = df["pct_change"].diff()
df = add_lagged_features(df, ["y_original"], lags=[1, 6, 12])
df = add_rolling_features(df, "y_original", windows=[3, 6, 12])
df.dropna(inplace=True)

# === CCF SELECTION ===
df_numeric = df.select_dtypes(include=[np.number]).copy()
ccf_ranked = rank_features_ccf(df_numeric, targetCol="y_original")
selected_features = [col for col in list(ccf_ranked[:TOP_K_FEATURES]) if col in df.columns]
features = df[selected_features]
target_log = np.log1p(df["y_original"])

# === SCALE ===
features_scaled_list, x_scaler = sklearn_fit_transform(features, StandardScaler())
target_scaled_list, y_scaler = sklearn_fit_transform(target_log.to_frame(), StandardScaler())

X_scaled = features_scaled_list[0].values
y_scaled = target_scaled_list[0].values.flatten()

# === SEQUENCES ===
X_seq, y_seq = create_direct_delta_sequences(X_scaled, y_scaled, SEQ_LEN, HORIZON)
X_seq = X_seq.reshape(X_seq.shape[0], SEQ_LEN, -1)
y_seq = y_seq.reshape(y_seq.shape[0], HORIZON)

# === SPLIT ===
val_split = int(len(X_seq) * 0.8)
X_train, X_val = X_seq[:val_split], X_seq[val_split:]
y_train, y_val = y_seq[:val_split], y_seq[val_split:]

train_loader = prepare_dataloader(X_train, y_train, batch_size=BATCH_SIZE)
val_loader = prepare_dataloader(X_val, y_val, shuffle=False, batch_size=BATCH_SIZE)


2025-04-04 19:37:50,211 - INFO - Added time features: year, month, quarter. DataFrame shape: (408, 363)


In [13]:
# === MODEL ===
model = LSTM(input_size=X_seq.shape[2], output_size=HORIZON).to(DEVICE)
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# === TRAIN LOOP ===
print("\n Training LSTM with Delta Learning...")
best_loss = float("inf")
epochs_no_improve = 0

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(DEVICE)
        y_batch = y_batch.to(DEVICE)
        optimizer.zero_grad()
        pred = model(X_batch)
        loss = criterion(pred, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    avg_train_loss = train_loss / len(train_loader)

    # === Validation ===
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(DEVICE)
            y_batch = y_batch.to(DEVICE)
            pred = model(X_batch)
            val_loss += criterion(pred, y_batch).item()
    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch+1:02d}: Train MAE={avg_train_loss:.6f} | Val MAE={avg_val_loss:.6f}")

    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        best_model_state = model.state_dict()
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE:
            print("Early stopping.")
            break

model.load_state_dict(best_model_state)

# === FINAL FORECAST (add deltas to base value) ===
model.eval()
with torch.no_grad():
    x_input = torch.tensor(X_seq[-1][np.newaxis], dtype=torch.float32).to(DEVICE)
    pred_delta = model(x_input).cpu().numpy().flatten()
    base_val = y_scaled[-1]
    pred_scaled = base_val + pred_delta

y_pred_rescaled = y_scaler.inverse_transform(pred_scaled.reshape(-1, 1)).flatten()
y_pred_final = np.expm1(y_pred_rescaled)


 Training LSTM with Delta Learning...
Epoch 01: Train MAE=0.381685 | Val MAE=0.175143
Epoch 02: Train MAE=0.301909 | Val MAE=0.144671
Epoch 03: Train MAE=0.250947 | Val MAE=0.150709
Epoch 04: Train MAE=0.214598 | Val MAE=0.132989
Epoch 05: Train MAE=0.175730 | Val MAE=0.084105
Epoch 06: Train MAE=0.142522 | Val MAE=0.091752
Epoch 07: Train MAE=0.108456 | Val MAE=0.087438
Epoch 08: Train MAE=0.083856 | Val MAE=0.083011
Epoch 09: Train MAE=0.063647 | Val MAE=0.059622
Epoch 10: Train MAE=0.048567 | Val MAE=0.066273
Epoch 11: Train MAE=0.043053 | Val MAE=0.065587
Epoch 12: Train MAE=0.038577 | Val MAE=0.053362
Epoch 13: Train MAE=0.036905 | Val MAE=0.052874
Epoch 14: Train MAE=0.034734 | Val MAE=0.053327
Epoch 15: Train MAE=0.032161 | Val MAE=0.052069
Epoch 16: Train MAE=0.030443 | Val MAE=0.044928
Epoch 17: Train MAE=0.029013 | Val MAE=0.047525
Epoch 18: Train MAE=0.027545 | Val MAE=0.046967
Epoch 19: Train MAE=0.027640 | Val MAE=0.049984
Epoch 20: Train MAE=0.026611 | Val MAE=0.062664
E

In [14]:
# === SAVE ===
save_path = os.path.join("..", "..", "Predictions", "LSTM.npy")
os.makedirs(os.path.dirname(save_path), exist_ok=True)
np.save(save_path, y_pred_final)
print(f"\nSaved improved LSTM with residuals to: {save_path}")


Saved improved LSTM with residuals to: ../../Predictions/LSTM.npy
